In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from torchvision.datasets import ImageFolder
import torchvision.transforms as transform
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
torch.utils.data.Dataset
import torchvision
import pathlib
from torch.optim import Adam
import glob
from torch.autograd import Variable

In [99]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [100]:
device

device(type='cuda')

In [101]:
epochs=5
batch_size=5
learning_rate=0.001

In [174]:
transformer=transform.Compose([
    transform.Resize((150,150)),
    transform.RandomHorizontalFlip(),
    transform.ToTensor(),
    transform.Normalize([0.5,0.5,0.5],
                       [0.5,0.5,0.5])
])

In [175]:
data_path='C:/Users/paltu/anaconda jp/eye'

In [176]:
dataset=ImageFolder(data_path,transform=transformer)

In [177]:
root=pathlib.Path(data_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [178]:
classes

['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

In [179]:
train_idx=[]
test_idx=[]

In [180]:
class_indices=dataset.targets
cataract_idx=[idx for idx,label in enumerate(class_indices) if label==0]
diabetic_idx=[idx for idx,label in enumerate(class_indices) if label==1]
glucomia_idx=[idx for idx,label in enumerate(class_indices) if label==2]
normal_idx=[idx for idx,label in enumerate(class_indices) if label==3]

In [181]:
print(len(cataract_idx+diabetic_idx+glucomia_idx+normal_idx))

4217


In [182]:
train_idx.extend(cataract_idx[:int(0.8*len(cataract_idx))])
train_idx.extend(diabetic_idx[:int(0.8*len(diabetic_idx))])
train_idx.extend(glucomia_idx[:int(0.8*len(glucomia_idx))])
train_idx.extend(normal_idx[:int(0.8*len(normal_idx))])

In [183]:
test_idx.extend(cataract_idx[int(0.8*len(cataract_idx)):])
test_idx.extend(diabetic_idx[int(0.8*len(diabetic_idx)):])
test_idx.extend(glucomia_idx[int(0.8*len(glucomia_idx)):])
test_idx.extend(normal_idx[int(0.8*len(normal_idx)):])

In [184]:
train_dataset=Subset(dataset,train_idx)
test_dataset=Subset(dataset,test_idx)

In [185]:
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=16,shuffle=True)

In [186]:
len(train_dataset)

3372

In [187]:
len(test_dataset)

845

In [188]:
3372+845

4217

In [189]:
def wt(w,f,p,s):
    return ((w-f+2*p)/s)+1

In [190]:
wt(75,3,1,1)

75.0

In [191]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=4):
        super(ConvNet,self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(num_features=20)
        self.relu2=nn.ReLU()
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        self.fc=nn.Linear(in_features=75*75*32,out_features=num_classes)
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        
        output=output.view(-1,32*75*75)
        output=self.fc(output)
        
        return output
        

In [192]:
model=ConvNet(num_classes=4).to(device)

In [193]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [194]:
num_epochs=30

In [195]:
train_count=len(train_dataset)
test_count=len(test_dataset)

In [196]:
train_count+test_count

4217

In [198]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for images,labels in train_loader:
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint4.model')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(5.2655) Train Accuracy: 0.608540925266904 Test Accuracy: 0.7088757396449704
Epoch: 1 Train Loss: tensor(1.4962) Train Accuracy: 0.7161921708185054 Test Accuracy: 0.5775147928994083
Epoch: 2 Train Loss: tensor(0.7102) Train Accuracy: 0.7986358244365361 Test Accuracy: 0.6343195266272189
Epoch: 3 Train Loss: tensor(0.4466) Train Accuracy: 0.8493475682087782 Test Accuracy: 0.727810650887574
Epoch: 4 Train Loss: tensor(0.3996) Train Accuracy: 0.8567615658362989 Test Accuracy: 0.7834319526627219
Epoch: 5 Train Loss: tensor(0.3191) Train Accuracy: 0.8819691577698695 Test Accuracy: 0.7834319526627219
Epoch: 6 Train Loss: tensor(0.2339) Train Accuracy: 0.9074733096085409 Test Accuracy: 0.7727810650887574
Epoch: 7 Train Loss: tensor(0.1760) Train Accuracy: 0.9338671411625148 Test Accuracy: 0.7195266272189349
Epoch: 8 Train Loss: tensor(0.1799) Train Accuracy: 0.930011862396204 Test Accuracy: 0.7917159763313609
Epoch: 9 Train Loss: tensor(0.1528) Train Accuracy: 0.9374

In [197]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for images,labels in train_loader:
        if torch.cuda.is_available():
            images = images.to(device)
            labels = labels.to(device)
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint4.model')
        best_accuracy=test_accuracy


AttributeError: 'int' object has no attribute 'to'